In [1]:
import pandas as pd
from tqdm import tqdm

pd.set_option('max_colwidth', None)
pd.set_option('display.max_columns', None)
from crossref_commons.retrieval import get_publication_as_json
from crossref_commons.iteration import iterate_publications_as_json

from utils import remove_html_tags, split_entries_in_column, extract_FKZ

/Users/isaacduong/BMBF_Clustering_Project/BMBFenv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
BMBF = pd.read_csv("../data/cleanedBMBF.csv")
BMBF.head(2)

,FKZ,Ressort,Referat,PT,Arb.-Einh.,Zuwendungsempfänger,Gemeindekennziffer,Stadt/Gemeinde,Ort,Bundesland,Staat,Ausführende Stelle,Gemeindekennziffer.1,Stadt/Gemeinde.1,Ort.1,Bundesland.1,Staat.1,Thema,Leistungsplansystematik,Klartext Leistungsplansystematik,Laufzeit von,Laufzeit bis,Fördersumme in EUR,Förderprofil,Verbundprojekt,Förderart
0,03F0212C/7,BMBF,724,PT-J,MGS1,Universität Rostock,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,Universität Rostock - Fachbereich Elektrotechnik und Informationstechnik,13003000.0,Rostock,Rostock,Mecklenburg-Vorpommern,Deutschland,"Verbundprojekt MESSIN: Entwicklung und Systemintegration der Komponenten Navigation, Automatische Steuerung, Kommunikation und Energieversorgung für den Meßdelphin MESSIN",FB4010,"Technik-/Infrastrukturentwicklung und -bereitstellung (Überwachungssysteme, Begleitforschung zum Monitoring, innovative Technik und Geräteträger)",01.01.1998,31.08.2000,"648.462,00",Forschung und Entwicklung zur Daseinsvorsorge,MESSIN,PDIR
1,D251800/0,BMBF,314,BIBB,A3.3,"Ministerium für Wirtschaft, Arbeit und Wohnungsbau Baden-Württemberg",8111000.0,Stuttgart,Stuttgart,Baden-Württemberg,Deutschland,KACO new energy GmbH,8121000.0,Heilbronn,Heilbronn,Baden-Württemberg,Deutschland,Prozeßorientierte Entwicklungsplanung und Qualifizierung im Betrieb,OB1050,Versuchs- und Modelleinrichtungen und -programme im Bereich der beruflichen Bildung,01.07.1997,30.06.2000,"96.634,00",Forschung und Entwicklung zur Daseinsvorsorge,NaN,PDIR


### At CROSSREF, each funder is assigned to a so-called funder number, which is unique and is used to identify the oganisation/ funder . So we will retrieve all metadatas which are related to the funder of each value in column "Ressort" as below. To do this, we make use of the python modul crossref-commons (https://api.crossref.org/swagger-ui/index.html)

## BMBF : 501100002347 Bundesministerium für Bildung und Forschung 
## BMWi : 501100002765 Bundesministerium für Wirtschaft und Technologie
## BMU : 501100013549 Bundesministerium für Umwelt, Naturschutz, nukleare Sicherheit und Verbraucherschutz
## BMVI : 100008383  Bundesministerium für Verkehr und Digitale Infrastruktur
## BMEL : 501100020631 Bundesministerium für Verbraucherschutz, Ernährung und Landwirtschaft
BMJV_BLE : no funder number found

In [168]:
funders = {"BMBF":"501100002347", "BMWi":"501100002765", "BMU":"501100013549", "BMVI":"100008383", "BMEL":"501100020631"}

In [169]:
#  50135 search results on 15.11.2023, which has funder "Bundesministerium für Bildung und Forschung BMBF"
def retrieve_metadata(funder, max_results = 50200):
    """ function to retrieve metadatas from publication funded by BMBF """
    
    
    metadatas = []
    filters = {'funder': funder}
    for i, p in tqdm(enumerate(iterate_publications_as_json(max_results=max_results, filter=filters))):
        pub = {}
        for f in p.get('funder'):
            if f.get('DOI') == f'10.13039/{funder}':
                pub['AWARD'] = f.get('award','')
                break
        pub["FUNDER"] = funder
        pub['DOI'] = p.get('DOI','')
        pub['TITLE'] = p.get('title','')
        pub['ABSTRACT'] = p.get('abstract','')
        pub['LANG'] = p.get('language','')
        
        if ('resource' in p.keys()) and ('primary' in p['resource'].keys()):
            pub['RESOURCE'] = p['resource']['primary'].get('URL','')
        else:
            pub['RESOURCE']=''

        metadatas.append(pub)
 
    return metadatas


In [173]:
def create_metadatas_table(funders):
    
    total_metadatas = []
    for funder, funder_number in funders.items():
        datas = retrieve_metadata(funder=funder_number)
        total_metadatas.extend(datas)

    return pd.DataFrame.from_records(total_metadatas)

In [ ]:
metadatas = create_metadatas_table(funders)

In [60]:
metadatas.to_csv("../data/crossref_pub_metadatas.csv",index = False)

In [174]:
# 51953 publications in total 
metadatas.shape

(52371, 7)

In [175]:
# but only 325 publications in German language
metadatas[metadatas["LANG"]=="de"].shape

(328, 7)

In [176]:
de_metadatas = metadatas[metadatas["LANG"]=="de"]


In [177]:
# and only 254 records have values in column "AWARD", which represent the FKZs we need to match the enrichment 
de_metadatas[de_metadatas["AWARD"] !=''].shape

(254, 7)

In [178]:
de_metadatas = de_metadatas[de_metadatas["AWARD"] !='']


In [179]:
de_metadatas["ABSTRACT"] = de_metadatas["ABSTRACT"].apply(remove_html_tags)

In [180]:
de_metadatas["AWARD"] = de_metadatas["AWARD"].apply(lambda li: li[0]) 
de_metadatas.AWARD

77                                                     01JKL19MET
149      Förderkennzeichen 01PL12043; Förderkennzeichen 01PB14014
174                                                     03XP0106C
354                                                BMBF, 01GY1323
375                                           02L18B570–02L18B575
                                   ...                           
51070                                                    03SIN540
51084                                                     325414B
51167                                       ENPRO2 (FKZ 03ET1517)
51677                                                28-V-9102X20
52311                                                  01MM19010B
Name: AWARD, Length: 254, dtype: object

In [148]:
de_metadatas = split_entries_in_column(de_metadatas, "AWARD")
de_metadatas

,AWARD,FUNDER,DOI,TITLE,ABSTRACT,LANG,RESOURCE
77,01JKL19MET,501100002347,10.1007/s11618-023-01144-0,"[Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen – ein systematischer Literaturüberblick, Overall conditions and challenges of arts and cultural education in rural areas—a systematic review]","<jats:title>Zusammenfassung</jats:title><jats:p>Kulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen.</jats:p>",de,https://link.springer.com/10.1007/s11618-023-01144-0
149,Förderkennzeichen 01PL12043,501100002347,10.1007/s11618-021-00993-x,[Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung],"<jats:title>Zusammenfassung</jats:title><jats:p>Die hochschuldidaktische Qualifizierung von Lehrenden hat insbesondere mit dem Bund-Länder-Programm Qualitätspakt Lehre und den in diesem Kontext entstandenen Angeboten an Bedeutung gewonnen. Zum Transfer des in den Qualifizierungen Gelernten in den Lehralltag und seinen Bedingungen liegen jedoch nur wenige Befunde vor. Der vorliegende Aufsatz stellt ein im Anschluss an den Diskurs der Lerntransferforschung entwickeltes und auf den deutschsprachigen, hochschuldidaktischen Bereich zugeschnittenes Instrument zur Erfassung von Lerntransferbedingungen vor, das die Dimensionen Transfermotivation, Selbstwirksamkeitserwartung, persönliche Transferkapazitäten, Trainingsdesign, Lehrbedingungen, Möglichkeit zur Anwendung, positive Folgen, Unterstützung durch Kollegen, Unterstützung durch Vorgesetzten, Transferklima und Unterstützung durch Studierende erfasst. Die auf konfirmatorischen Faktorenanalysen basierenden Auswertungen geben Hinweise auf die Konstruktvalidität des Instruments, das sowohl zur Evaluation des Lerntransfers und seinen Bedingungen im Kontext hochschuldidaktischer Weiterbildungen als auch zur Weiterentwicklung des diesbezüglichen Forschungsstandes geeignet ist.</jats:p>",de,https://link.springer.com/10.1007/s11618-021-00993-x
149,Förderkennzeichen 01PB14014,501100002347,10.1007/s11618-021-00993-x,[Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung],"<jats:title>Zusammenfassung</jats:title><jats:p>Die hochschuldidaktische Qualifizierung von Lehrenden hat insbesondere mit dem Bund-Länder-Programm Qualitätspakt Lehre und den in diesem Kontext entstandenen Angeboten an Bedeutung gewonnen. Zum Transfer des in den Qualifizierungen Gelernten in den Lehralltag und seinen Bedingungen liegen jedoch nur wenige Befunde vor. Der vorliegende Aufsatz stellt ein im Anschluss an den Diskurs der Lerntransferforschung entwickeltes und auf den deutschsprachigen, hochschuldidaktischen Bereich zugeschnittenes Instrument zur Erfassung von Lerntransferbedingungen vor, das die Di

In [149]:
de_metadatas["AWARD"].shape

(263,)

In [150]:
# extract FKZ from word such es Förderkennzeichen
de_metadatas["AWARD"] = de_metadatas["AWARD"].apply(extract_FKZ)


In [151]:
# enpack list column TITLE  
de_metadatas["TITLE"] = de_metadatas["TITLE"].apply( lambda li: li[0])


In [152]:
# join both columns TITLE and ABSTRACT 
de_metadatas.loc[:,"crossref_enrichment"] = de_metadatas.loc[:,"TITLE"] + " " + de_metadatas.loc[:,"ABSTRACT"] 
de_metadatas

,AWARD,FUNDER,DOI,TITLE,ABSTRACT,LANG,RESOURCE,crossref_enrichment
77,01JKL19MET,501100002347,10.1007/s11618-023-01144-0,Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen – ein systematischer Literaturüberblick,"<jats:title>Zusammenfassung</jats:title><jats:p>Kulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen.</jats:p>",de,https://link.springer.com/10.1007/s11618-023-01144-0,"Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen – ein systematischer Literaturüberblick <jats:title>Zusammenfassung</jats:title><jats:p>Kulturelle Bildung ist auf eine vielfältige Angebotsstruktur und die Kooperationen unterschiedlicher Bildungs- und Kultureinrichtungen angewiesen, um kulturelle Teilhabe zu ermöglichen. Die intensive Förderung durch öffentliche, zivilgesellschaftliche und private Akteur*innen hat in den letzten Jahren zu einem Ausbau von formalen und non-formalen Bildungsangeboten geführt. Gleichzeitig finden sich zunehmend Forschungsarbeiten auf dem Gebiet, deren Fokus weiterhin stärker auf städtische Bildungskontexte als auf die Bedingungen ländlicher Räume gerichtet ist. Vor allem fehlt bislang eine systematische Bestandsaufnahme der vorhandenen Studien und Erkenntnisse zu kultureller Bildung speziell in ländlichen Räumen. Der vorliegende systematische Review setzt hier an und erarbeitet einen umfassenden Überblick über Publikationen zu kultureller Bildung, Kulturarbeit und Soziokultur in ländlichen Räumen. Inhaltlich steht bei dieser Untersuchung die Aufarbeitung der Rahmenbedingungen und Herausforderungen kultureller Bildung in ländlichen Räumen im Mittelpunkt. Die Beschreibung zentraler Themen erfolgt dabei entlang von drei Teilbereichen: kommunal- und kulturpolitischen Bedingungen, (infra-) strukturellen Faktoren sowie kulturellen und gesellschaftlichen Dimensionen.</jats:p>"
149,01PL12043,501100002347,10.1007/s11618-021-00993-x,Ein Instrument zur Messung von Lerntransferbedingungen im Kontext der hochschuldidaktischen Weiterbildung,"<jats:title>Zusammenfassung</jats:title><jats:p>Die hochschuldidaktische Qualifizierung von Lehrenden hat insbesondere mit dem Bund-Länder-Programm Qualitätspakt Lehre und den in diesem Kontext entstandenen Angeboten an Bedeutung gewonnen. Zum Transfer des in den Qualifizierungen Gelernten in den Lehralltag und seinen Bedingungen liegen jedoch nur wenige Befunde vor. Der vorliegende Aufsatz stellt ein im Anschluss an den Diskurs der Lerntransferforschung entwickeltes und auf den deutschsprachigen, hochschuldidaktischen Bereich zugeschnittenes Instrument zur Erfassung von Lerntransferbedingungen vor, das die Dimensionen Transfermotivation, Selbstwirksamkeitserwartung, persönliche Transferkapazitäten, Trainingsdesign, Lehrbedingungen, Mögli

In [153]:
# duplicated in "AWARD" columns indicates that there are more than 1 publication/ press release for some topics
de_metadatas.AWARD.value_counts()

AWARD
FKZ           6
BMBF          5
01JA1621      3
03SFK1C0      2
01EL1824A     2
             ..
16FWN009/1    1
01EA1411A     1
01UW0808      1
21INVI06      1
01MM19010B    1
Name: count, Length: 237, dtype: int64

In [181]:
de_metadatas.shape

(254, 7)

In [155]:
# create a dataframe, which merge all publications / press releases with same funder number.
crossref_enrichment = de_metadatas.groupby('AWARD')['crossref_enrichment'].agg(lambda x: ' '.join(x)).reset_index()


In [182]:
crossref_enrichment

,AWARD,crossref_enrichment
0,01E,Beeinflussende Faktoren bei der Inanspruchnahme von Hilfen
1,01EA1411A,Die familiäre Essumgebung im ersten Lebensjahr
2,01EC1404E,Trittsicher durchs Leben: Analyse von 1092 Bewegungskursen im ländlichen Raum
3,01EH1101B,Informelles Pflegepotenzial bei Älteren in Deutschland
4,01EL1420A,SMARTMOVE – Einbezug von Familien in die Entwicklung und Implementierung digitaler Gesundheitsangebote
...,...,...
232,W142400B,"“Something is building up, there is someone who is reliable and who has money” – Actor constellations, coordination of action and educational output in fragile organizational settings. The example of work-oriented basic education <jats:title>Zusammenfassung</jats:title><jats:p>Der Beitrag untersucht anhand der theoretischen Ansätze der<jats:italic>Educational Governance</jats:italic>und des Neo-Institutionalismus die Steuerung in dem, bezüglich Finanzierung und Angebotsentwicklung fragilen Feld der arbeitsorientierten Grundbildung. Dabei wird fokussiert, wie soziale Ordnungen über die Koordination von Handlungen in verschiedenen Akteurskonstellationen pädagogische Leistung möglich machen. Konkret geht es darum, herauszuarbeiten, welche institutionellen und organisationalen Strukturen in der arbeitsorientierten Grundbildung erkennbar sind und welche Formen der Handlungskoordination zwischen Akteuren welche pädagogischen Leistungen zustande kommen lassen. Empirische Befunde weisen auf ein differenziertes und unterschiedlich dichtes Mehrebenensystem hin, in dem die Akteure über unterschiedliche Formen der Handlungskoordination Leistungen sichern und weiterentwickeln.</jats:p>"
233,W145800,"Strukturierte Kommunikation als Modus agiler Organisation: Eine Fallstudie zu Gruppenprozessen und Lernen bei einem Finanzdienstleister <jats:title>Zusammenfassung</jats:title><jats:p>Die in der Zeitschrift „Gruppe. Interaktion. Organisation (GIO)“ erschienene Fallbeschreibung zeigt auf, wie strukturierte Feedbackprozesse in Teams eine fachliche und persönliche Weiterentwicklung ermöglichen. Dazu werden die Arbeitsweisen und Teamprozesse eines Start-up-Unternehmens, das IT-basierte Finanzdienstleistungen anbietet, betrachtet. Diese in der Praxis herausgebildeten und stetig weiterentwickelten Formate werden empirisch rekonstruiert und mit dem „Agile Framework“ verglichen. Es zeigt sich, dass diese Praktiken den Grundsätzen agilen Arbeitens entsprechen und Bedingungen für ein Lernen im Arbeitsprozess, den Aufbau von Wissen und eine flexible und anpassungsfähige Organisation gewährleisten. Abschließend wird diskutiert, wie Führung und Organisationen vor diesem Hintergrund die gegensätzlichen Anforderungen an Stabilität und Agilität austarieren können.</jats:p>"
234,W1473AFO,"Schreibcoaching im beruflichen Übergangssystem. Auf der Suche nach Förderbedarfen und wirksamen Gelingensfaktoren <jats:title>Zusammenfassung</jats:title><jats:p>Der folgende Beitrag präsentiert das sprachdidaktisch-sozialwissenschaftliche Projekt KOFISCH, in dem ein empirie-basiertes Konzept zur Schreibförderung im beruflichen Übergangssystem für junge Erwachsene mit geringer Literalität entwickelt wird. Basis des Konzepts sind Ergebnisse aus qualitativen empirischen Untersuchungen in einer berufsvorbereitenden Bildungsmaßnahme mit produktionsorientiertem Ansatz. Diese ergeben sich zum einen aus Hospitationen und Interviews mit Anleiter*innen (<jats:italic>n</jats:italic> = 4), Sozialpädagog*innen (<jats:italic>n</jats:italic> = 2) und Lehrkräften (<jats:italic>n</jats:italic> = 3) zu Handlungsabläufen und Schreibanforderungen und zum anderen aus Textbewertungen authentischer Lernertexte (<jats:italic>n</jats:italic> = 208) zur Erhebung der vorhandenen Kompetenzen sowie Förderbedarfe der jugendlichen Teilnehmenden (<jats:italic>n</jats:italic> = 24; 23 männlich und 1 weiblich im Alter von 16 bis 19 Jahren) im Bereich der berufsbezogenen Schreibkompetenz. Das daraus resultierende Förderkonzept beruht auf den authentischen Schreibanforde

In [10]:
crossref_enrichment.to_csv("../data/BMBF_crossref_enrichment.csv",index=False)


In [6]:
crossref_enrichment["crossref_enrichment"] = crossref_enrichment["crossref_enrichment"].apply(remove_html_tags)

In [8]:
crossref_enrichment["crossref_enrichment"] = crossref_enrichment["crossref_enrichment"].apply(lambda text: text.replace("\n",""))